In [3]:
!pip install pydub

In [4]:
import os
import librosa
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from pydub import AudioSegment
import pandas as pd
from sklearn.metrics import classification_report

In [5]:
TRAIN_DIR = '/content/drive/MyDrive/train'
TEST_DIR = '/content/drive/MyDrive/test'

In [6]:
CLASSES = sorted([cls for cls in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, cls))])
print("Classes and Labels:")
for idx, class_name in enumerate(CLASSES):
    print(f"Class: {class_name}, Label: {idx}")

Classes and Labels:
Class: 0, Label: 0
Class: 1, Label: 1
Class: 2, Label: 2
Class: 3, Label: 3
Class: 4, Label: 4
Class: 5, Label: 5
Class: 6, Label: 6
Class: 7, Label: 7
Class: 8, Label: 8
Class: 9, Label: 9
Class: none, Label: 10


In [7]:
def audio_to_melspectrogram(file_path, max_len=128):
    audio, sr = librosa.load(file_path, sr=16000)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

    if log_mel_spec.shape[1] > max_len:
        log_mel_spec = log_mel_spec[:, :max_len]
    else:
        log_mel_spec = np.pad(log_mel_spec, ((0, 0), (0, max_len - log_mel_spec.shape[1])), mode='constant')

    return log_mel_spec

In [8]:
def convert_to_wav(file_path):
    if file_path.lower().endswith(".mp3"):
        wav_path = file_path.rsplit(".", 1)[0] + ".wav"
        audio = AudioSegment.from_mp3(file_path)
        audio.export(wav_path, format="wav")
        print(f"Converted {file_path} to {wav_path}")
        return wav_path
    return file_path

In [9]:
class AudioDataset(Dataset):
    def __init__(self, data_dir, classes, max_len=128, transform=None):
        self.file_paths = []
        self.labels = []
        self.max_len = max_len
        self.transform = transform
        self.classes = classes

        for label, class_name in enumerate(classes):
            class_dir = os.path.join(data_dir, class_name)
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                file_path = convert_to_wav(file_path)  # بررسی و تبدیل mp3 به wav
                self.file_paths.append(file_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        mel_spec = audio_to_melspectrogram(file_path, max_len=self.max_len)

        if self.transform:
            mel_spec = self.transform(mel_spec)

        return mel_spec, label, os.path.basename(file_path)

In [10]:
train_dataset = AudioDataset(TRAIN_DIR, CLASSES, transform=ToTensor())
test_dataset = AudioDataset(TEST_DIR, CLASSES, transform=ToTensor())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [11]:
class AudioClassifier(nn.Module):
    def __init__(self, num_classes):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AudioClassifier(num_classes=len(CLASSES)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [15]:
from tqdm import tqdm

best_accuracy = 0.0
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    print(f"Epoch {epoch + 1}/{num_epochs}")
    progress_bar = tqdm(train_loader, desc="Training", unit="batch")

    for mel_specs, labels, _ in progress_bar:
        mel_specs = mel_specs.to(device).unsqueeze(1).squeeze(2)
        labels = labels.to(device)

        outputs = model(mel_specs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar.set_postfix({"Loss": loss.item(), "Accuracy": 100. * correct / total})

    accuracy = 100. * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs} Completed, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_model.pth")
        print(f"New Best Model Saved with Accuracy: {best_accuracy:.2f}%")


Epoch 1/40


Training: 100%|██████████| 677/677 [02:39<00:00,  4.24batch/s, Loss=1.69, Accuracy=37.9]


Epoch 1/40 Completed, Loss: 1.6986, Accuracy: 37.88%
New Best Model Saved with Accuracy: 37.88%
Epoch 2/40


Training: 100%|██████████| 677/677 [02:28<00:00,  4.55batch/s, Loss=1.58, Accuracy=42]


Epoch 2/40 Completed, Loss: 1.5997, Accuracy: 41.97%
New Best Model Saved with Accuracy: 41.97%
Epoch 3/40


...

Epoch 38/40 Completed, Loss: 0.1329, Accuracy: 95.87%
Epoch 39/40


Training: 100%|██████████| 677/677 [02:32<00:00,  4.45batch/s, Loss=0.0261, Accuracy=95.6]


Epoch 39/40 Completed, Loss: 0.1293, Accuracy: 95.63%
Epoch 40/40


Training: 100%|██████████| 677/677 [02:34<00:00,  4.40batch/s, Loss=0.0542, Accuracy=96.2]

Epoch 40/40 Completed, Loss: 0.1197, Accuracy: 96.24%
New Best Model Saved with Accuracy: 96.24%


In [16]:
model.eval()
all_preds = []
all_labels = []
file_names = []

with torch.no_grad():
    for mel_specs, labels, files in test_loader:
        mel_specs = mel_specs.to(device).unsqueeze(1).squeeze(2)
        labels = labels.to(device)

        outputs = model(mel_specs)
        _, predicted = outputs.max(1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        file_names.extend(files)

In [ ]:
output_df = pd.DataFrame({
    0: file_names,
    1: [CLASSES[pred] for pred in all_preds]})
output_df.to_csv("model_output.csv", index=False, header=False)


In [18]:
torch.save(model.state_dict(), "audio_classifier.pth")

In [ ]:
torch.save(model.state_dict(), "best_model.pth")